In [31]:
import json
import re
import requests
from subprocess import run
from collections import defaultdict
import runpy
import tempfile

In [4]:
data = json.loads(requests.get('https://storage.googleapis.com/flutter_infra/releases/releases_linux.json').text)
flutter_versions = data['releases']

In [53]:
# replace these
flutter_repo_path = '../co/sdk/flutter'
engine_repo_path = '../co/sdk/engine'
dart_repo_path = '../co/sdk/dart-sdk/full-clone'

def make_cache(fn):
    d = {}
    def caching_fn(x):
        if x not in d:
            d[x] = fn(x)
        return d[x]
    return caching_fn

def get_engine_version(flutter_commit):
    p = run(['git', 'show', flutter_commit + ':bin/internal/engine.version'], check=True, capture_output=True, cwd=flutter_repo_path)
    return p.stdout.decode().strip()

def get_dart_version(engine_commit):
    contents = run(['git', 'show', engine_commit + ':DEPS'], check=True, capture_output=True, cwd=engine_repo_path).stdout
    m = re.search(r"'dart_revision': '([a-fA-F0-9]+)'", contents.decode())
    return m.group(1)

sv_template_file = tempfile.NamedTemporaryFile()
sv_template_file.write('{{SNAPSHOT_HASH}}'.encode())
sv_template_file.flush()
sv_out_file = tempfile.NamedTemporaryFile()

def get_snapshot_hash(dart_commit):
    run(['git', 'checkout', dart_commit], check=True, cwd=dart_repo_path)
    sv_out_file.seek(0)
    run(['python2', 'tools/make_version.py', '--input=' + sv_template_file.name, '--output=' + sv_out_file.name], check=True, cwd=dart_repo_path)
    return sv_out_file.read().decode()

get_engine_version = make_cache(get_engine_version)
get_dart_version = make_cache(get_dart_version)
get_snapshot_hash = make_cache(get_snapshot_hash)

#get_engine_version('master')
#get_dart_version('11d756a62ed0ddf87a9ce20b219b55300ec6b67d')
#get_snapshot_version('06536d68ca0f27528b0bf729f4b8d673ed14beda')

In [58]:
print('| Release date | Channel | Version | Commit | Engine commit | Dart SDK commit | Snapshot version |')
print('| ------------ | ------- | ------- | ------ | ------------- | --------------- | ---------------- |')
for v in sorted(flutter_versions, key=lambda x: x['release_date'], reverse=True):
    if v['channel'] not in { 'stable', 'beta' }: continue
    engine_commit = get_engine_version(v['hash'])
    dart_commit = get_dart_version(engine_commit)
    snapshot_hash = get_snapshot_hash(dart_commit)
    print(f"| {v['release_date'][:10]} | {v['channel']} | {v['version']} | {v['hash']} | {engine_commit} | {dart_commit} | {snapshot_hash} |")

| Release date | Channel | Version | Commit | Engine commit | Dart SDK commit | Snapshot version |
| ------------ | ------- | ------- | ------ | ------------- | --------------- | ---------------- |
| 2020-10-08 | stable | 1.22.1 | f30b7f4db93ee747cd727df747941a28ead25ff5 | 75bef9f6c8ac2ed4e1e04cdfcd88b177d9f1850d | efd753621946a89008b76b76d85d54d1aa57fce8 | 8ee4ef7a67df9845fba331734198a953 |
| 2020-10-08 | beta | 1.22.0-12.4.pre | f30b7f4db93ee747cd727df747941a28ead25ff5 | 75bef9f6c8ac2ed4e1e04cdfcd88b177d9f1850d | efd753621946a89008b76b76d85d54d1aa57fce8 | 8ee4ef7a67df9845fba331734198a953 |
| 2020-10-01 | stable | 1.22.0 | d408d302e22179d598f467e11da5dd968dbdc9ec | 5babba6c4d25fa237bbf755ab85c9a0c50b3c6ec | 41eab9b49ccce8960f71c657861dc629f96295af | 8ee4ef7a67df9845fba331734198a953 |
| 2020-09-29 | beta | 1.22.0-12.3.pre | d408d302e22179d598f467e11da5dd968dbdc9ec | 5babba6c4d25fa237bbf755ab85c9a0c50b3c6ec | 41eab9b49ccce8960f71c657861dc629f96295af | 8ee4ef7a67df9845fba331734198a953 |
